In [1]:
import flask

In [2]:
import requests
from io import StringIO
from datetime import datetime
import time
from dateutil.relativedelta import relativedelta
import pandas as pd
import json


# 參考 twstock 取得需要的 URL
SESSION_URL = 'http://mis.twse.com.tw/stock/index.jsp'
STOCKINFO_URL = 'http://mis.twse.com.tw/stock/api/getStockInfo.jsp?ex_ch={stock_id}&_={time}'


def get_realtime_quote(stockNo, ex='tse'):
    req = requests.Session()
    req.get(SESSION_URL)
    
    stock_id = '{}_{}.tw'.format(ex, stockNo)

    r = req.get(STOCKINFO_URL.format(stock_id=stock_id, time=int(time.time()) * 1000))

    try:
        return r.json()
    except json.decoder.JSONDecodeError:
        return {'rtmessage': 'json decode error', 'rtcode': '5000'}


def convert_date(x):
    year, month, day = x.split('/')
    return datetime(int(year)+1911, int(month), int(day))


def get_tw_stock_df(stockNo, date):
    url = "http://www.twse.com.tw/exchangeReport/STOCK_DAY?response=csv&date={}&stockNo={}"
    df = pd.read_csv(url.format(date.strftime("%Y%m%d"), stockNo), skipfooter=4, skiprows=1)
    # 因為 .csv 擋後面有多一個 comma，所以會產生一個叫做 Unnamed 的 column
    return df.loc[:, ~df.columns.str.contains("Unnamed")]


# def get_tw_stock(stockNo, date):
#     url = "http://www.twse.com.tw/exchangeReport/STOCK_DAY"
#     params = {}
#     params['stockNo'] = stockNo
#     params['date'] = date.strftime("%Y%m%d")
#     params['response'] = "csv"
#     r = requests.get(url, params=params)
#     df = pd.read_csv(StringIO(r.text), skiprows=1, skip_footer=4, thousands=',')
#     df['日期'] = df['日期'].apply(convert_date)
#     return df.loc[:, ~df.columns.str.contains("Unnamed")].set_index('日期')

def get_tw_stock(stockNo, date):
    url = "http://www.twse.com.tw/exchangeReport/STOCK_DAY"
    params = {}
    params['stockNo'] = stockNo
    params['date'] = date.strftime("%Y%m%d")
    data = requests.get(url, params=params).json()
    df = pd.DataFrame(data['data'], columns=data['fields'])
    df['日期'] = df['日期'].apply(convert_date)
    df['成交股數'] = pd.to_numeric(df['成交股數'].str.replace(",",""))
    df['成交金額'] = pd.to_numeric(df['成交金額'].str.replace(",",""))
    df['成交筆數'] = pd.to_numeric(df['成交筆數'].str.replace(",",""))
    df['開盤價'] = pd.to_numeric(df['開盤價'])
    df['最高價'] = pd.to_numeric(df['最高價'])
    df['最低價'] = pd.to_numeric(df['最低價'])
    df['收盤價'] = pd.to_numeric(df['收盤價'])
    return df.set_index('日期')


def get_historical_data(stockNo, start, end=None, delay=3):
    if not end:
        end = datetime.now()
    data = []
    while start <= end:
        data.append(get_tw_stock(stockNo, start))
        time.sleep(delay)
        start += relativedelta(months=1)
    return pd.concat(data)


def get_data_months(stockNo, months=3, delay=3):
    now = datetime.now()
    data = []
    while months > 0:
        data.append(get_tw_stock(stockNo, now))
        time.sleep(delay)
        now -= relativedelta(months=1)
        months -= 1
    return pd.concat(data).sort_index()


if __name__=="__main__":
    # df = get_tw_stock("2330", datetime(2016, 1, 1))
    # print(df.head())
    #df = get_historical_data("2330", datetime(2017, 8, 1))
    df = get_data_months("2330", months=2, delay=3)
    print(df)
    

                 成交股數         成交金額    開盤價    最高價    最低價    收盤價    漲跌價差   成交筆數
日期                                                                           
2020-10-05   33238832  14450407588  438.0  438.0  431.5  432.5   -0.50  18148
2020-10-06   36276395  15935558262  441.5  442.0  437.5  439.5   +7.00  20189
2020-10-07   44691239  19693987093  435.0  445.0  435.0  443.0   +3.50  24438
2020-10-08   60853811  27461867933  450.0  454.0  448.5  453.0  +10.00  37378
2020-10-12   67645223  31061721030  458.0  462.0  457.5  460.0   +7.00  36118
2020-10-13   46303738  21346346714  465.0  465.0  456.5  462.0   +2.00  25938
2020-10-14   52677241  24157138176  460.5  461.0  457.0  459.0   -3.00  22599
2020-10-15   49987278  22668836399  456.0  456.5  450.0  453.0   -6.00  23794
2020-10-16   48274801  21775749575  451.0  456.0  448.5  449.0   -4.00  26602
2020-10-19   34846906  15854021670  453.0  457.5  451.0  457.5   +8.50  17112
2020-10-20   26567638  12065806730  455.5  457.0  451.0  451.0  